# Рекомендательная система

Задача - написать систему, способную предсказывать пользователю фильмы, которые он оценит. Это эквивалентно задаче ранжирования имеющигося наборов фильмов, так, чтобы прежде всего выдавались самые интересные среди еще не просмотренных фильмов. Для решения использовал:
<ul>
    <li><b>Факторизацию</b>, то есть разложил матрицу пользователь-товар на две матрицы, содержащие скрытые признаки предпочтений пользователей и особенностей товаров. Использовал SVD-разложение в разных реализациях</li>
    <li><b>Контентный подход</b> - сделал из описаний фильмов от пользователей TF-IDF эмбеддинги, а затем измерял степень похожести полученных текстов</li>
</ul>
Среди метрик выбрал среди Precision at k (p@k), MGG, nDCG, pFound. Остановился на первой - она самая простая и при этом не учитывает порядок рекомендаций. Упор решил сделать именно на наличие релевантных фильмов в выдаче алгоритма, их порядок оставил в стороне.

In [2]:
from scipy.sparse import csc_matrix
from scipy import sparse
import pandas as pd
import numpy as np

In [3]:
rates = pd.read_csv('rating.csv')

In [4]:
rates.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [5]:
users_number = rates.userId.unique().shape[0]
movies_number = rates.movieId.unique().shape[0]
users_number, movies_number

(138493, 26744)

In [6]:
id_to_row = { dfid: row for row, dfid in enumerate(rates.userId.unique()) }
id_to_col = { dfid: col for col, dfid in enumerate(rates.movieId.unique()) }

In [7]:
prepared_data = rates.drop(columns='timestamp')
prepared_data['userId'] = np.array([id_to_row[dfid] for dfid in rates.userId])
prepared_data['movieId'] = np.array([id_to_col[dfid] for dfid in rates.movieId])
prepared_data

,userId,movieId,rating
0,0,0,3.5
1,0,1,3.5
2,0,2,3.5
3,0,3,3.5
4,0,4,3.5
...,...,...,...
20000258,138492,1814,4.5
20000259,138492,1037,4.5
20000260,138492,3950,3.0
20000261,138492,1818,5.0


In [8]:
user_film_matrix = csc_matrix((prepared_data['rating'].values, (prepared_data['userId'].values.astype(int), prepared_data['movieId'].values.astype(int))))

Подготовил датасет с рейтингами, сделал разреженную сатрицу пользователь-продукт.

In [9]:
user_film_matrix

<138493x26744 sparse matrix of type '<class 'numpy.float64'>'
	with 20000263 stored elements in Compressed Sparse Column format>

In [48]:
from scipy.linalg import svd

svd(user_film_matrix.todense())

MemoryError: Unable to allocate 27.6 GiB for an array with shape (138493, 26744) and data type float64

Как и следовало ожидать, данные огромны и занимают много памяти, использовать svd из scipy не выйдет.

In [12]:
!pip install cython
!pip install sparsesvd

<a href="https://pypi.org/project/sparsesvd/">Свд на разреженной матрице</a>

In [13]:
from sparsesvd import sparsesvd

ut, s, vt = sparsesvd(user_film_matrix, 5)

Беру f=5. Получаю 5 признаков товара и 5 предпочтений пользователя. Умножая матрицу призаков товаров на матрицу предпочтений данного пользователя, получаю числа, по которым могу ранжировать.

In [16]:
from sklearn.decomposition import TruncatedSVD

svd_users = TruncatedSVD(n_components=10, n_iter=10)
svded_users = svd_users.fit(user_film_matrix)

In [111]:
svded_users.transform(user_film_matrix[0])

array([[20.03771706, -4.45696616,  0.99440048, -3.59568524, -9.06578936,
         3.36720137,  2.05124509,  6.07699847, -5.36178934,  1.4623455 ,
        -1.58243568,  1.37672644, -3.26066888, 10.11393113,  1.18601043,
         5.34172206,  1.26297806,  0.25700577,  2.17136259, -2.80654454,
        -5.33845419,  2.63715366,  1.35774994,  0.5595823 ,  3.75674578]])

In [114]:
svd_films = TruncatedSVD(n_components=25, n_iter=7, random_state=42)
svded_films = svd_films.fit(user_film_matrix.T)

In [115]:
svded_films.transform(user_film_matrix.T[0])

array([[259.9862393 ,  95.73593357, 120.13639078, -74.22653025,
        -28.67424554,  16.14964747, -64.03243882, -49.42486402,
        -18.55344069,   7.30076608, -26.69836222,  26.44873469,
        -44.40279859, -38.94945599,  -5.17088289, -22.70884276,
        -22.84441425, -14.41338467,  -5.66448593, -34.57429122,
         19.0548537 , -42.27354191, -25.89360131, -11.54128998,
         25.09036274]])

In [123]:
svded_users.transform(user_film_matrix[0]) @ svded_films.transform(user_film_matrix.T[4]).T / 4000 * 5

array([[16.75136297]])